In [1]:
import os, sys
from selectionStrategies1p import *
from selectionStrategies2p import *
from playSingle import *
from play2players import *
from diceGameFunctions import *

In [2]:
play_dice_game_1p(ss1p_12,rounds = 100)

Player averaged 12.86 throws for 100 rounds


In [3]:
play_dice_game_2p(ss2p_a,ss2p_5,rounds = 1000)

Player 1 wins 435 vs. 352 and 213 draws.


In [4]:
import pandas as pd

sid_df = pd.read_csv('StrategyPerSID.csv')
sid_df = sid_df.fillna('nan')
sid_df.head()

,sid,name,ss1p,ss2p,is1p,is2p,always_p2,correct_sol,nice_sol
0,107062421,ta,ss1p_a,ss2p_1,nan,nan,nan,nan,nan
1,108031227,林子瑜,ss1p_11,ss2p_4,nan,nan,nan,nan,nan
2,108020011,鍾允睿,ss1p_a,ss2p_a,nan,nan,nan,True,nan
3,108072108,林淇臻,ss1p_a,ss2p_a,nan,nan,nan,nan,nan
4,108031221,潘昱銘,ss1p_a,ss2p_6,nan,nan,nan,nan,nan


In [5]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import os, sys\nfrom selectionStrategies1p import *\nfrom selectionStrategies2p import *\nfrom playSingle import *\nfrom play2players import *\nfrom diceGameFunctions import *',
  'play_dice_game_1p(ss1p_12,rounds = 100)',
  'play_dice_game_2p(ss2p_a,ss2p_5,rounds = 1000)',
  "import pandas as pd\n\nsid_df = pd.read_csv('StrategyPerSID.csv')\nsid_df = sid_df.fillna('nan')\nsid_df.head()",
  'globals()'],
 '_oh': {4:          sid name     ss1p    ss2p is1p is2p always_p2 correct_sol nice_sol
  0  107062421   ta   ss1p_a  ss2p_1  nan  nan       nan         nan      nan
  1  108031227  林子瑜  ss1p_11  ss2p_4  nan  nan       nan         nan      nan
  2  108020011  鍾允睿   ss1p_a  ss2p_a  nan  nan       nan        True

In [6]:
def play_dice_game_1p_sid(sid,rounds=100):
    '''given sd play n rounds single player game'''
    
    #Raise exception
    if sid not in sid_df['sid'].values:
        print("Unable to call function, ckeck SIDs")
        return
        
    ss1p = sid_df.loc[sid_df['sid']==sid,'ss1p'].values[0]
    if ss1p == 'nan':
        ss1p = ss1p_r
    else:
        ss1p = globals()[ss1p]

    play_dice_game_1p(ss1p,rounds)
    
def play_dice_game_2p_sid(sid1,sid2,rounds=100):
	'''given sid1 and sid2 play n rounds 2 player game'''
	
	#Raise exception
	if sid1 not in sid_df['sid'].values: 
		print("Unable to call function, ckeck SIDs")
		return
	elif sid2 not in sid_df['sid'].values:
		print("Unable to call function, ckeck SIDs")
		return

	#p1
	ss2p_p1 = sid_df.loc[sid_df['sid']==sid1,'ss2p'].values[0]
	if ss2p_p1 == 'nan':
		ss2p_p1 = ss2p_r
	else:
		ss2p_p1 = globals()[ss2p_p1]
		
	#p2
	ss2p_p2 = sid_df.loc[sid_df['sid']==sid2,'ss2p'].values[0]
	if ss2p_p2 == 'nan':
		ss2p_p2 = ss2p_r
	else:
		ss2p_p2 = globals()[ss2p_p2]
	
	#if
	if sid_df.loc[sid_df['sid']==sid1,'always_p2'].values[0] == True and \
		sid_df.loc[sid_df['sid']==sid2,'always_p2'].values[0] == True:
		raise Exception
	elif sid_df.loc[sid_df['sid']==sid1,'always_p2'].values[0] == True:
		print('reordering players...')
		tempss = ss2p_p1
		ss2p_p1 = ss2p_p2
		ss2p_p2 = tempss
		
	play_dice_game_2p(ss2p_p1,ss2p_p2,rounds)   

In [7]:
def play_dice_game_1p(selection_strategy, rounds=100):
	'''play dice game, 1 player. 
	INPUT: selection_strategy,initialization_strategy, rounds
	OUTPUT: average No. of throws over rounds rounds
	'''
	dice_throws_arr = []

	for i in range(0,rounds):
		
		#vars
		profit = 0
		throws = 0
		win = 0
		fail = 0
		count_A = 0
		count_B = 0
		count_C = 0
		prev_action = is_r() #choose a random action to begin
		prev_dice_res = random.randint(1,6)

		while profit < GOAL:
			
			#sel action, roll dice, count reward
			kwargs = {'profit':profit,'throws':throws, 'prev_action':prev_action,\
					 'prev_dice_res':prev_dice_res, 'win':win, 'count_A':count_A,
					 'count_B':count_B,'count_C':count_C}
			sel_action = selection_strategy(**kwargs)

			#counters
			if sel_action == action_A:
				count_A += 1
			elif sel_action == action_B:
				count_B+=1
			elif sel_action == action_C:
				count_C+=1
			else:
				raise 

			dice_res = random.randint(1,6)
			if dice_res in sel_action['match']:
				win += 1
				profit += sel_action['reward']
			
			#update state
			throws += 1
			prev_action = sel_action
			prev_dice_res = dice_res

		dice_throws_arr.append(throws)
	
	avg = np.mean(dice_throws_arr)
	print('Player averaged '+ str(avg) + ' throws for '+ str(rounds) +' rounds')

def play_dice_game_2p(selection_strategy_p1,selection_strategy_p2,
	rounds=100):
	'''play dice game, 2 players. 
	INPUT: selection_strategy,initialization_strategy, rounds
	OUTPUT: winner array containing 1 for p1 win, 2 for p2 win and 0 for draw
	'''
	winner_array = []

	for i in range(0,rounds):
		profit_p1 = 0
		profit_p2 = 0
		win_p1 = 0
		win_p2 = 0
		throws = 0

		prev_action_p1 = is_r()  #choose a random action to begin
		prev_action_p2 = is_r() #choose a random action to begin
		prev_dice_res = -1

		while profit_p1 < GOAL and profit_p2 < GOAL:
			#select strategy
			args_p1 = {'profit_p1':profit_p1,'profit_p2':profit_p2,\
					  'throws':throws, 'prev_action_p1':prev_action_p1,\
					  'prev_action_p2':prev_action_p2,\
					  'prev_dice_res':prev_dice_res, \
					  'win_p1':win_p1,'win_p2':win_p2,'whoAmI':'p1',\
					  }
			sel_action_p1 = selection_strategy_p1(**args_p1)
			args_p2 = {'profit_p1':profit_p1,'profit_p2':profit_p2,\
					  'throws':throws, 'prev_action_p1':prev_action_p1,\
					  'prev_action_p2':prev_action_p2,\
					  'prev_dice_res':prev_dice_res, \
					  'win_p1':win_p1,'win_p2':win_p2,'whoAmI':'p2',\
					  'sel_action_p1':sel_action_p1}
			sel_action_p2 = selection_strategy_p2(**args_p2)

			#check reward
			dice_res = random.randint(1,6)
			if dice_res in sel_action_p1['match']:
				profit_p1 += sel_action_p1['reward']
			if dice_res in sel_action_p2['match']:
				profit_p2 += sel_action_p2['reward']

			#update
			throws += 1
			prev_action_1 = sel_action_p1
			prev_action_2 = sel_action_p2
			prev_dice_res = dice_res
		
		if profit_p1==profit_p2:
			win_p1 += 1
			win_p2 += 1
			winner_array.append(0)
		elif profit_p1>profit_p2:
			win_p1 += 1
			winner_array.append(1)
		elif profit_p1<profit_p2:
			win_p2 += 1
			winner_array.append(2)
			
	#decide winner
	winner_array = np.array(winner_array)
	chooseWinner(winner_array,rounds)

def chooseWinner(winner_arr,rounds):
	
	p2 = len(np.where(winner_arr==2)[0])
	p1 = len(np.where(winner_arr==1)[0])
	draws = rounds - p1 - p2
	
	if p1==p2:
		print('Draw between player 1 and 2')

	elif p1>p2:
		print('Player 1 wins ' + str(p1) + ' vs. ' + str(p2) +' and ' + str(draws) + ' draws.')

	elif p2>p1:
		print('Player 2 wins ' + str(p2) + ' vs. ' + str(p1) +' and ' + str(draws) + ' draws.')


In [8]:
#test 1 p strategies
for sid in sid_df['sid'].values:
    print(sid)
    play_dice_game_1p_sid(sid,rounds=10000)
    print('*****')

107062421
Player averaged 11.9533 throws for 10000 rounds
*****
108031227
Player averaged 12.0342 throws for 10000 rounds
*****
108020011
Player averaged 11.9512 throws for 10000 rounds
*****
108072108
Player averaged 11.9257 throws for 10000 rounds
*****
108031221
Player averaged 11.8994 throws for 10000 rounds
*****
108006212
Player averaged 12.0493 throws for 10000 rounds
*****
108072112
Player averaged 11.9425 throws for 10000 rounds
*****
108006219
Player averaged 11.9845 throws for 10000 rounds
*****
108006229
Player averaged 12.0997 throws for 10000 rounds
*****
106032091
Player averaged 13.0405 throws for 10000 rounds
*****
108072107
Player averaged 12.0243 throws for 10000 rounds
*****
108072226
Player averaged 11.8983 throws for 10000 rounds
*****
108061237
Player averaged 11.9097 throws for 10000 rounds
*****
108061132
Player averaged 12.2891 throws for 10000 rounds
*****
107042025
Player averaged 12.0376 throws for 10000 rounds
*****
108061251
Player averaged 11.9619 throws

In [9]:
#test 2p strategies
TA_sid = 107062421
for sid in sid_df['sid'].values:
    print(sid)
    play_dice_game_2p_sid(TA_sid,sid,rounds=1000)
    print('*****')

107062421
Draw between player 1 and 2
*****
108031227
Player 1 wins 551 vs. 449 and 0 draws.
*****
108020011
Player 1 wins 435 vs. 271 and 294 draws.
*****
108072108
Player 1 wins 438 vs. 236 and 326 draws.
*****
108031221
Player 1 wins 567 vs. 433 and 0 draws.
*****
108006212
Player 1 wins 462 vs. 252 and 286 draws.
*****
108072112
Player 1 wins 561 vs. 439 and 0 draws.
*****
108006219
Player 1 wins 507 vs. 387 and 106 draws.
*****
108006229
Player 1 wins 436 vs. 250 and 314 draws.
*****
106032091
Player 1 wins 558 vs. 364 and 78 draws.
*****
108072107
Player 1 wins 545 vs. 455 and 0 draws.
*****
108072226
Player 1 wins 540 vs. 460 and 0 draws.
*****
108061237
Player 1 wins 422 vs. 261 and 317 draws.
*****
108061132
Player 1 wins 555 vs. 445 and 0 draws.
*****
107042025
Player 1 wins 526 vs. 474 and 0 draws.
*****
108061251
Player 1 wins 432 vs. 283 and 285 draws.
*****
108006219
Player 1 wins 526 vs. 363 and 111 draws.
*****
108020018
Player 1 wins 429 vs. 262 and 309 draws.
*****
10

In [10]:
win_sid = [106041010,106031122,107061150,108062262]
#ss2p_19,ss2p_16,ss2p_37,ss2p_38

In [11]:
for sid in win_sid:
    print('sid: %d'%sid)
    
    for i in range(0,5):
        play_dice_game_2p_sid(sid,TA_sid,rounds=1000)

sid: 106041010
Player 2 wins 496 vs. 270 and 234 draws.
Player 2 wins 488 vs. 275 and 237 draws.
Player 2 wins 492 vs. 292 and 216 draws.
Player 2 wins 526 vs. 254 and 220 draws.
Player 2 wins 498 vs. 276 and 226 draws.
sid: 106031122
Draw between player 1 and 2
Draw between player 1 and 2
Draw between player 1 and 2
Draw between player 1 and 2
Draw between player 1 and 2
sid: 107061150
Player 2 wins 489 vs. 405 and 106 draws.
Player 2 wins 470 vs. 407 and 123 draws.
Player 2 wins 484 vs. 408 and 108 draws.
Player 2 wins 475 vs. 408 and 117 draws.
Player 2 wins 486 vs. 411 and 103 draws.
sid: 108062262
Player 2 wins 607 vs. 393 and 0 draws.
Player 2 wins 586 vs. 414 and 0 draws.
Player 2 wins 600 vs. 400 and 0 draws.
Player 2 wins 598 vs. 402 and 0 draws.
Player 2 wins 585 vs. 415 and 0 draws.


In [12]:
for sid in win_sid:
    print('sid: %d'%sid)
    
    for i in range(0,5):
        play_dice_game_2p_sid(sid,108062262,rounds=1000)

sid: 106041010
Player 2 wins 542 vs. 458 and 0 draws.
Player 2 wins 546 vs. 454 and 0 draws.
Player 2 wins 570 vs. 430 and 0 draws.
Player 2 wins 530 vs. 470 and 0 draws.
Player 2 wins 526 vs. 474 and 0 draws.
sid: 106031122
Player 1 wins 590 vs. 410 and 0 draws.
Player 1 wins 598 vs. 402 and 0 draws.
Player 1 wins 610 vs. 390 and 0 draws.
Player 1 wins 587 vs. 413 and 0 draws.
Player 1 wins 610 vs. 390 and 0 draws.
sid: 107061150
Player 1 wins 570 vs. 425 and 5 draws.
Player 1 wins 536 vs. 463 and 1 draws.
Player 1 wins 556 vs. 442 and 2 draws.
Player 1 wins 538 vs. 461 and 1 draws.
Player 1 wins 533 vs. 466 and 1 draws.
sid: 108062262
Player 2 wins 502 vs. 492 and 6 draws.
Player 1 wins 530 vs. 468 and 2 draws.
Player 1 wins 508 vs. 492 and 0 draws.
Player 2 wins 505 vs. 492 and 3 draws.
Player 1 wins 523 vs. 473 and 4 draws.


In [13]:
#106031122  always wins

In [14]:
play_dice_game_2p(ss2p_a,ss2p_1,rounds=1000)

Player 2 wins 446 vs. 262 and 292 draws.


In [15]:
for i in range(0,5):
    play_dice_game_1p(ss1p_a,rounds=10)

Player averaged 9.6 throws for 10 rounds
Player averaged 12.5 throws for 10 rounds
Player averaged 13.3 throws for 10 rounds
Player averaged 5.6 throws for 10 rounds
Player averaged 12.8 throws for 10 rounds


In [16]:
for i in range(0,5):
    play_dice_game_1p(ss1p_b,rounds=10)

Player averaged 10.7 throws for 10 rounds
Player averaged 11.9 throws for 10 rounds
Player averaged 13.0 throws for 10 rounds
Player averaged 10.2 throws for 10 rounds
Player averaged 12.4 throws for 10 rounds


In [17]:
for i in range(0,5):
    play_dice_game_1p(ss1p_1,rounds=10)

Player averaged 12.5 throws for 10 rounds
Player averaged 16.4 throws for 10 rounds
Player averaged 13.0 throws for 10 rounds
Player averaged 8.9 throws for 10 rounds
Player averaged 11.7 throws for 10 rounds


In [18]:
for i in range(0,5):
    play_dice_game_1p(ss1p_c,rounds=100)

Player averaged 11.9 throws for 100 rounds
Player averaged 12.07 throws for 100 rounds
Player averaged 12.16 throws for 100 rounds
Player averaged 12.27 throws for 100 rounds
Player averaged 12.02 throws for 100 rounds


In [19]:
#test 2p strategies
TA_sid = 107062421
for sid in sid_df['sid'].values:
    print(sid)
    play_dice_game_2p_sid(TA_sid,sid,rounds=1000)
    print('*****')

107062421
Draw between player 1 and 2
*****
108031227
Player 1 wins 544 vs. 456 and 0 draws.
*****
108020011
Player 1 wins 460 vs. 247 and 293 draws.
*****
108072108
Player 1 wins 439 vs. 263 and 298 draws.
*****
108031221
Player 1 wins 557 vs. 443 and 0 draws.
*****
108006212
Player 1 wins 436 vs. 243 and 321 draws.
*****
108072112
Player 1 wins 564 vs. 436 and 0 draws.
*****
108006219
Player 1 wins 526 vs. 361 and 113 draws.
*****
108006229
Player 1 wins 449 vs. 255 and 296 draws.
*****
106032091
Player 1 wins 551 vs. 392 and 57 draws.
*****
108072107
Player 1 wins 536 vs. 464 and 0 draws.
*****
108072226
Player 1 wins 550 vs. 450 and 0 draws.
*****
108061237
Player 1 wins 423 vs. 251 and 326 draws.
*****
108061132
Player 1 wins 572 vs. 428 and 0 draws.
*****
107042025
Player 2 wins 509 vs. 491 and 0 draws.
*****
108061251
Player 1 wins 437 vs. 240 and 323 draws.
*****
108006219
Player 1 wins 523 vs. 385 and 92 draws.
*****
108020018
Player 1 wins 454 vs. 246 and 300 draws.
*****
108

In [20]:

def ss2p_48(**kwargs):
	''' B until profit==6, A otherwiaw ....'''
	whoAmI = kwargs['whoAmI']
	if whoAmI == 'p1':
		profit_own = kwargs['profit_p1']
		profit_opp = kwargs['profit_p2']
	else:
		profit_own = kwargs['profit_p2']
		profit_opp = kwargs['profit_p1']

	if profit_own==0:
		return action_B
	elif profit_own==2:
		return action_C
	elif profit_own==3:
		return action_B
	elif profit_own==4:
		return action_C
	elif profit_own==5:
		return action_C
	elif profit_own==6:
		return action_A
	elif profit_own==7:
		return action_C
	elif profit_own==8:
		return action_C
	elif profit_own==9:
		return action_B
	elif profit_own==10:
		return action_C

In [21]:
for i in range(0,5):
    play_dice_game_2p(ss2p_48,ss2p_1,rounds=1000)

Draw between player 1 and 2
Draw between player 1 and 2
Draw between player 1 and 2
Draw between player 1 and 2
Draw between player 1 and 2


In [23]:
for i in range(0,5):
    play_dice_game_1p(ss1p_1,rounds=100)

Player averaged 11.66 throws for 100 rounds
Player averaged 13.09 throws for 100 rounds
Player averaged 11.59 throws for 100 rounds
Player averaged 11.35 throws for 100 rounds
Player averaged 12.93 throws for 100 rounds
